In [83]:

#only salary results you will get are high salaries

#date are too far apart (only take in 2020)
#lease_commence has better variable , remaining lease (remove lease_commencence)
#unrelated (remove block)
#street name has better variable, town_name (remoce street_name)
#data leakage?
#missing data
#misimput
#duplicates

#preprocessing
#categorical. onehot and ordinality
#numerical, remaining_lease


#prices not real


In [84]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import re

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_percentage_error

In [85]:
data = pd.read_csv('Flat_prices_noexecutives.csv')
data = pd.DataFrame(data)

data = data.drop_duplicates(keep='first')
data = data.drop(['remaining_lease','lease_commence_date','storey_range','block','month','flat_type'],axis = 1)
data = data.dropna(axis=0)

print(data.columns)
print(data.dtypes)
data

Index(['town', 'floor_area_sqm', 'flat_model', 'resale_price',
       'remaining_lease_years', 'storey_min', 'flat_type.1'],
      dtype='object')
town                      object
floor_area_sqm           float64
flat_model                object
resale_price             float64
remaining_lease_years      int64
storey_min                 int64
flat_type.1              float64
dtype: object


,town,floor_area_sqm,flat_model,resale_price,remaining_lease_years,storey_min,flat_type.1
0,ANG MO KIO,73.0,New Generation,265000.0,55,4,3.0
1,ANG MO KIO,70.0,Model A,470000.0,91,19,3.0
2,ANG MO KIO,73.0,New Generation,230000.0,56,1,3.0
3,ANG MO KIO,73.0,New Generation,280000.0,55,4,3.0
4,ANG MO KIO,68.0,New Generation,220000.0,59,7,3.0
...,...,...,...,...,...,...,...
28003,YISHUN,122.0,Improved,440000.0,65,4,5.0
28004,YISHUN,122.0,Improved,467000.0,64,7,5.0
28005,YISHUN,122.0,Improved,508000.0,64,10,5.0
28006,YISHUN,122.0,Improved,450000.0,66,1,5.0


In [86]:
datax = data.copy()
datax = datax.drop(['resale_price'],axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(datax,data.resale_price)


categorical_column = [col for col in datax.columns if datax[col].dtypes == 'object']
numerical_column = [n for n in datax.columns if datax[n].dtypes == 'int64' or datax[n].dtypes == 'float64']

cols = numerical_column + categorical_column
X_train = X_train[cols]
X_valid = X_valid[cols]
print(X_train)

       floor_area_sqm  remaining_lease_years  storey_min  flat_type.1  \
25948            59.0                     55          13          3.0   
11615            46.0                     94           1          2.0   
26808           134.0                     62           7          5.0   
4718             60.0                     45           4          3.0   
435             121.0                     76           7          5.0   
...               ...                    ...         ...          ...   
2906             82.0                     55          13          4.0   
2978             67.0                     94           7          3.0   
24830           110.0                     82          13          5.0   
13404           119.0                     75           4          5.0   
27273           117.0                     52          22          5.0   

                town         flat_model  
25948          BEDOK           Improved  
11615  BUKIT PANJANG            Model A

In [87]:
for c in data.columns:
    if data[c].dtypes == 'object' :
        print(c,data[c].nunique())

town 26
flat_model 16


In [88]:

numpreprocessor = Pipeline(steps=[
    ('imp',SimpleImputer(strategy='constant'))
])

catpreprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numpreprocessor, numerical_column),
        ('cat', catpreprocessor, categorical_column)    
    ]
)

In [89]:

model = RandomForestRegressor(n_estimators=100, random_state=0)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_percentage_error(y_valid, preds)
print('MAE:', score)

MAE: 0.05929893421540797
